# 1.) Imports

In [ ]:
# Imports
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import StratifiedShuffleSplit
import nibabel as nib

In [ ]:
# Class
class BrainAgeDataset(Dataset):
    def __init__(self, metadata_df, nifti_dir, transform=None, age_weights=None, debug=False):
        """
        Args:
            metadata_df (DataFrame): Must contain 'SubjectID' and 'Age' columns.
            nifti_dir (str): Directory containing NIfTI files.
            transform (callable, optional): Optional transform to apply to image tensors.
            age_weights (dict, optional): Mapping from age bins to sample weights.
            debug (bool): If True, prints subject info during loading.
        """
        self.df = metadata_df.reset_index(drop=True)
        self.nifti_dir = nifti_dir
        self.transform = transform
        self.age_weights = age_weights
        self.debug = debug
        self.target_shape = (128, 128, 128)

        if self.age_weights:
            self.bins = list(age_weights.keys())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        subject_id = row['SubjectID']
        age = row['Age']

        # Load and normalize NIfTI image
        nifti_path = os.path.join(self.nifti_dir, f'{subject_id}_MR1.nii.gz')
        img = nib.load(nifti_path).get_fdata()
        img = np.squeeze(img)
        img = (img - np.mean(img)) / (np.std(img) + 1e-5)

        # Convert to tensor and resize
        img_tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0)  # [1, D, H, W]
        img_tensor = F.interpolate(img_tensor.unsqueeze(0), size=self.target_shape, mode='trilinear', align_corners=False).squeeze(0)

        if self.transform:
            img_tensor = self.transform(img_tensor)

        age_tensor = torch.tensor(age, dtype=torch.float32)

        if self.debug:
            print(f"[DEBUG] Subject: {subject_id} | Age: {age:.1f} | Image shape: {img_tensor.shape}")

        if self.age_weights:
            age_bin = pd.cut([age], bins=self.bins)[0]
            weight = self.age_weights.get(age_bin, 1.0)
            weight_tensor = torch.tensor(weight, dtype=torch.float32)
            return img_tensor, age_tensor, weight_tensor
        else:
            return img_tensor, age_tensor

# Data Augmentation
def brain_mri_augment(volume, flip_prob=0.5, noise_std=0.002):
    """
    Apply random flipping, noise, and intensity jitter to a 3D volume.
    """
    volume = volume.clone()

    # Random flipping
    if random.random() < flip_prob:
        volume = torch.flip(volume, dims=[1])
    if random.random() < flip_prob:
        volume = torch.flip(volume, dims=[2])
    if random.random() < flip_prob:
        volume = torch.flip(volume, dims=[3])

    # Gaussian noise
    volume += torch.randn_like(volume) * noise_std

    # Intensity jitter
    scale = random.uniform(0.95, 1.05)
    shift = random.uniform(-0.05, 0.05)
    volume = volume * scale + shift

    return volume

# Reproducibility
def set_seed(seed=42):
    """Set random seed for reproducibility across NumPy, PyTorch, and Python."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Model Utility
def count_parameters(model):
    """Return the number of trainable parameters in a model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Data Splitting
def split_dataframe(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
    """
    Randomly split a DataFrame into train/val/test sets.
    """
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-5
    df = df.sample(frac=1, random_state=seed).reset_index(drop=True)

    total = len(df)
    train_end = int(train_ratio * total)
    val_end = train_end + int(val_ratio * total)

    train_df = df.iloc[:train_end]
    val_df = df.iloc[train_end:val_end]
    test_df = df.iloc[val_end:]

    return train_df, val_df, test_df

def stratified_split(df, bins=8, train_size=0.7, val_size=0.15, test_size=0.15, seed=42):
    """
    Stratified split based on age bins to preserve age distribution across splits.
    """
    assert abs(train_size + val_size + test_size - 1.0) < 1e-5

    df = df.copy()
    df['age_bin'] = pd.qcut(df['Age'], q=bins, duplicates='drop')

    # First split: train vs temp
    sss1 = StratifiedShuffleSplit(n_splits=1, train_size=train_size, random_state=seed)
    train_idx, temp_idx = next(sss1.split(df, df['age_bin']))
    train_df = df.iloc[train_idx].reset_index(drop=True)
    temp_df = df.iloc[temp_idx].reset_index(drop=True)

    # Second split: val vs test
    sss2 = StratifiedShuffleSplit(n_splits=1, train_size=val_size / (val_size + test_size), random_state=seed)
    val_idx, test_idx = next(sss2.split(temp_df, temp_df['age_bin']))
    val_df = temp_df.iloc[val_idx].reset_index(drop=True)
    test_df = temp_df.iloc[test_idx].reset_index(drop=True)

    for d in [train_df, val_df, test_df]:
        d.drop(columns='age_bin', inplace=True)

    return train_df, val_df, test_df